In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pd_p

In [2]:
dataframe = pd.read_csv('dataset.csv', index_col=0)
dataframe.head()

,date,rConsumption,USGrowth,GDPDef,RFR,qt,year,GDPDef_p,rUSGrowth,rRFR
0,1971-01-01,3156.978,1.080000,22.391,3.840000,1,1971,NaN,NaN,NaN
1,1971-04-01,3185.993,1.136667,22.685,4.250000,2,1971,0.013130,1.121935,4.194920
2,1971-07-01,3211.394,1.113333,22.916,5.010000,3,1971,0.010183,1.102111,4.959498
3,1971-10-01,3264.679,1.113333,23.107,4.230000,4,1971,0.008335,1.104131,4.195035
4,1972-01-01,3307.767,1.240000,23.458,3.436667,1,1972,0.015190,1.221446,3.385244


In [17]:
def u(c, p):
    utility = c**(1-p['gamma'])/(1-p['gamma'])
    return utility

def u_p(c, p):
    utility_p = c**(-p['gamma'])
    return utility_p
    
def m(c0,c1,  p):
    m = p['beta'] * u_p(c1, p)/u_p(c0, p)
    return m

def get_m(dataset, params):
    c0 = dataset.rConsumption.to_numpy()[:-1]
    c1 = dataset.rConsumption.to_numpy()[1:]
    return m(c0,c1 ,params )

These exercises will give some familiarity with the GMM estimation of asset pricing models.
Assume that the joint distribution of $(m_{𝑡+1}, R_{t+1})$ is constant for all t (this would occur, for
example, if the stochastic process $\omega$ is Markov.) Define $m_{t+1} = \beta \frac{u'(c_{t+1})}{u'(c_t)}$ with $ u(c) = \frac{c^{1−\gamma}}{1-\gamma}$.

Download data on real personal consumption expenditures (FRED series PCECC96) and returns to the aggregate U.S. stock market (e.g. the Wilshire 5000 given in FRED series
      
WILL5000INDFC) as well as the risk-free rate (FRED series GDPDEF, the GDP deflator, is one possible measure of inflation). One possibility for data on the risk-free rate is the 3-month T-Bill rate given in FRED series TB3MS. Note that you will need to transform the Wilshire 5000 series and the T-Bill series into quarterly data as well as calculate the real returns to the Wilshire 5000 by accounting for inflation.

1. Assume that $\beta$ = 0.99 (since this is quarterly data, this implies an annual risk- free rate of about 4%). Calculate the sample analogs of the expected values and covariance in (5). Using scipy.optimize.broyden1, solve for the value of $\gamma$ that leads to equation (5) holding.

In [18]:
params = {'beta': 0.99, 'gamma':0}

In [19]:
def est_gamma_1(gamma, p):
    p['gamma'] =gamma
    m = get_m(dataframe, p)
    R = dataframe.rUSGrowth.to_numpy()[1:]
    
    cov = (m*R).mean()
    e = cov -1
    
    return e    

def est_gamma_2(gamma, p):
    p['gamma'] =gamma
    m = get_m(dataframe, p)
    r = dataframe.rRFR.to_numpy()[1:]
    
    e = 1/m - r -1
    
    return e    

In [20]:
from scipy.optimize import broyden1
F1 = lambda gamma: est_gamma_1(gamma, params)
broyden1(F1, 0.0001)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/nonlin.py:999: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)


NoConvergence: 43.159866566504554

In [22]:
F2 = lambda gamma: est_gamma_2(gamma, params)
broyden1(F1, 1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/optimize/nonlin.py:999: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)


NoConvergence: 44.17642168657913


2. How would you determine if this parameter is reasonable?

3. Now, consider the moment restriction given in equation (7). Continue to assume that $\beta$ = 0.99. Estimate the $\gamma$ that generates the SDF using only this moment condition. How does this $\gamma$ differ from the estimate of $\gamma$ found previously?

4. Finally, estimate $\beta$ and $\gamma$ using both equation (5) and equation (7). You will need to specify a weighting matrix (the identity matrix is a great place to start). Plot the objective function for different $(\beta, \gamma)$ pairs. How does your estimation differ from the previous two ways of estimating those parameters?


Note. The idea behind these problems comes from Hansen and Singleton (1982), although the implementation has some differences. For a survey of this literature, see Campbell (2003).